In [0]:
GALAXY = 'umi'

OBS_PATH = '/datascope/subaru/data/cmdfit/dSph'
HSC_FILE = '/datascope/subaru/data/cmdfit/dSph/umi_tpall3e_g24.cat'
GAIA_FILE = '/datascope/subaru/data/cmdfit/dSph/gaia.h5'
SKY_FILE = '/datascope/subaru/data/cmdfit/dSph/sky_ursaminor.feather'
FLUXSTD_FILE = '/datascope/subaru/data/cmdfit/dSph/fluxstd_ursaminor.feather'
MLCLASS_FILE = '/datascope/subaru/data/targeting/dSph/umi/ursaminor_mlclass.csv'
PMAP_FILE = '/datascope/subaru/data/cmdfit/run/umi/sim/nobin_chab_nb_250k_001/pmap.h5'
OUTPUT_PATH = '/datascope/subaru/user/dobos/netflow'
ISOCHRONES_PATH = '/datascope/subaru/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc'

GAIA_CROSSMATCH_RADIUS = 0.1    # in arcsec

NVISITS = 6
OUTPUT_PATH = f'/datascope/subaru/user/dobos/netflow/{GALAXY}_{NVISITS}_visit'

In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec
from scipy.special import logsumexp
from scipy.interpolate import interp1d

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.time import Time

In [0]:
plt.rc('font', size=6) #controls default text size

# Set up plotting

In [0]:
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow

In [0]:
galaxy = GALAXIES[GALAXY]
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

In [0]:
pointings = galaxy.get_pointings(SubaruPFI)
pointing = pointings[0]
pointings

In [0]:
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

# Load the data

In [0]:
from pfs.ga.targeting.data import Observation, Catalog
from pfs.ga.targeting.io import TextObservationReader, DataFrameSerializer
from pfs.ga.targeting.util.astro import *

In [0]:
# obs = SubaruHSC.text_observation_reader().read(HSC_FILE)
# obs.data.shape


# This could be done nicer with a FeatherObservationReader or similar
obs = Observation()
obs.append_photometry(SubaruHSC.photometry())
fn = os.path.join(OUTPUT_PATH, f'{GALAXY}_obs.feather')
obs._set_data(DataFrameSerializer().read(fn))
obs.data.shape

In [0]:
obs_mask = obs.data['priority'] > 0
obs_mask.shape, obs_mask.sum()

In [0]:
from pfs.ga.targeting.io import FeatherSkyReader, FeatherFluxStdReader

In [0]:
r = FeatherSkyReader()
sky = r.read(SKY_FILE)
sky.data.shape

In [0]:
r = FeatherFluxStdReader()
fluxstd = r.read(FLUXSTD_FILE)
fluxstd.data.shape

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)
# fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, obs, c='b')
fov.plot_observation(ax, sky, c='lightgray')
fov.plot_observation(ax, fluxstd, c='r')

pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })
for p in galaxy.get_pointings(SubaruPFI)[:]:
    pfi.plot_focal_plane(ax, fov, corners=True, projection=SubaruWFC(p))

ax.set_aspect('equal', adjustable='datalim')

# f.tight_layout()

# Load the assignments

In [0]:
from pfs.ga.targeting.io import DataFrameSerializer

fn = os.path.join(OUTPUT_PATH, f'{GALAXY}_assignments.feather')

s = DataFrameSerializer()
assignments = s.read(fn)

assignments

In [0]:
# Load the summary

fn = os.path.join(OUTPUT_PATH, f'{GALAXY}_summary.feather')

s = DataFrameSerializer()
summary = s.read(fn)

summary

In [0]:
assignments['RA']

In [0]:
# Plot focal plane and sky coordinates

# Exclude engineering and not-assigned fibers without coordinates
assignments = assignments[assignments['RA'] != 0]

f, axs = plt.subplots(1, 2, figsize=(6, 3), dpi=240)

axs[0].plot(assignments['fp_x'], assignments['fp_y'], 'o', ms=1, markeredgewidth=0)
axs[0].plot(assignments['fp_x'][100], assignments['fp_y'][100], 'or', ms=1, markeredgewidth=0)
axs[0].set_aspect('equal', adjustable='datalim')
axs[0].set_title("Focal plane")

axs[1].plot(assignments['RA'], assignments['Dec'], 'o', ms=1, markeredgewidth=0)
axs[1].plot(assignments['RA'][100], assignments['Dec'][100], 'or', ms=1, markeredgewidth=0)
# axs[1].set_aspect('equal', adjustable='datalim')
axs[1].set_title("Sky")

# Statistics

In [0]:
from pfs.datamodel import TargetType

In [0]:
# Number of unique calibration targets for each visit
assignments[assignments['prefix'] == 'cal'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# Number of unique sky positions for each visit
assignments[assignments['prefix'] == 'sky'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# Number of observed unique science targets for each visit
assignments[assignments['prefix'] == 'sci'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# Total number of unique science targets observed
assignments[assignments['prefix'] == 'sci']['targetid'].nunique()

In [0]:
# Targets that are partially observed
summary[(summary['num_visits'] < summary['req_visits']) & (summary['num_visits'] > 0)]

In [0]:
# Science targets that are observed longer than the required number of visits
summary[(summary['prefix'] == 'sci') & (summary['num_visits'] > summary['req_visits'])]

In [0]:
# Number of unique targets per priority class
summary[summary['num_visits'] > 0].groupby('class')['targetid'].nunique()

In [0]:
# Number of missed science targets per priority class
summary[(summary['prefix'] == 'sci') & (summary['num_visits'] == 0)].groupby('class')['targetid'].nunique()

In [0]:
# Number of partially observed science targets per priority class
summary[(summary['prefix'] == 'sci') & 
        (summary['num_visits'] < summary['req_visits']) & 
        (summary['num_visits'] > 0)].groupby('class')['targetid'].nunique()

In [0]:
# Number of unassigned fibers in each visit
assignments[assignments['target_type'] == TargetType.UNASSIGNED].groupby('visit_idx')['fiberid'].nunique()

In [0]:
np.sort(assignments['pointing_idx'].unique())

In [0]:
# Cobra assignments - target index for each cobra
cobra_assignments = {}
for pidx in np.sort(assignments['pointing_idx'].unique()):
    print(f'Pointing #{pidx}')
    cobra_assignments[pidx] = []
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        print(f'Visit #{vidx}')
        # Only the current visits and only and excl. engineering and empty fibers)
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx) & \
               (assignments['cobraid'] != -1)
        
        a = np.array(assignments[mask].sort_values('cobraid')[['targetid']])
        print(pidx, vidx, a.shape)

        cobra_assignments[pidx].append(a)

    cobra_assignments[pidx] = np.stack(cobra_assignments[pidx], axis=-1)

In [0]:
# Fibers that have moved to another target between visits
for pidx in np.sort(assignments['pointing_idx'].unique()):
    print(f'Pointing #{pidx}')
    
    cobra_moved = np.any(np.diff(np.stack(cobra_assignments[pidx], axis=-1), axis=-1) != 0, axis=-1)
    print(cobra_moved.shape)

    print('number of cobras moving between targets:', cobra_moved.sum())
    print('moving cobras:', np.where(cobra_moved)[0])

In [0]:
# of unique science targets assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx) & \
               (assignments['prefix'] == 'sci')
        print(f'Pointing #{pidx}, Visit #{vidx}:', assignments[mask]['targetid'].nunique())

print('Grand Total:', assignments[assignments['prefix'] == 'sci']['targetid'].nunique())

In [0]:
# total # of sky fibers assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx) & \
               (assignments['prefix'] == 'sky')
        print(f'Pointing #{pidx}, Visit #{vidx}:', assignments[mask]['targetid'].nunique())

print('Grand Total:', assignments[assignments['prefix'] == 'sky']['targetid'].nunique())

In [0]:
# total # of calibration fibers assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx) & \
               (assignments['prefix'] == 'cal')
        print(f'Pointing #{pidx}, Visit #{vidx}:', assignments[mask]['targetid'].nunique())

print('Grand Total:', assignments[assignments['prefix'] == 'cal']['targetid'].nunique())

# Plot cobra assignments

In [0]:
# Get a mask of HSC objects that are targeted

# The Netflow class actually has a function to return these but we
# don't want to spin up Netflow in this notebook

obs_assignments = {}
obs_assigned = None

sky_assignments = {}
fluxstd_assignments = {}

for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx)
        
        a = obs.data[['objid']].join(assignments[mask].set_index('targetid'), on='objid')
        a = np.array(~pd.isna(a['fiberid']))
        obs_assignments[vidx] = a
        obs_assigned = a if obs_assigned is None else obs_assigned | a

        a = sky.data[['skyid']].join(assignments[mask].set_index('targetid'), on='skyid')
        a = np.array(~pd.isna(a['fiberid']))
        sky_assignments[vidx] = a

        a = fluxstd.data[['objid']].join(assignments[mask].set_index('targetid'), on='objid')
        a = np.array(~pd.isna(a['fiberid']))
        fluxstd_assignments[vidx] = a

obs.data.shape, obs_assigned.shape, obs_assigned.sum()

In [0]:
# Plot distribution of priorities

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

hist = np.bincount(obs.data['priority'][obs_mask])
ax.bar(np.arange(hist.size - 1), hist[:-1])

hist = np.bincount(obs.data['priority'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)])
ax.bar(np.arange(hist.size), hist, color='r')


ax.set_title('Priority class number distribution')
ax.set_xlabel('Priority class')
ax.set_ylabel('Target count')


In [0]:
# Plot distribution of required visits

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

hist = np.bincount((np.ceil(obs.data['exp_time'][obs_mask & (obs.data['priority'][obs_mask] < 9)] / 1800.0)).astype(int))
ax.bar(np.arange(hist.size), hist)

hist = np.bincount((np.ceil(obs.data['exp_time'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)] / 1800.0)).astype(int))
ax.bar(np.arange(hist.size), hist, color='r')


ax.set_title('Distribution of required visit (t_exp = 1800 s)')
ax.set_xlabel('Number of required visits')
ax.set_ylabel('Target count')


In [0]:
# Make a plot that shows the difference of a single design vs moving fiber design

In [0]:
np.bincount(obs.data['exp_time'][(obs.data['priority'] >= 0) & (obs.data['priority'] < 9)] / 1800)

In [0]:
np.bincount(np.ceil(obs.data['exp_time'][(obs.data['priority'] >= 0) & (obs.data['priority'] < 9)] / 1800))

In [0]:
hist = np.bincount(np.ceil(obs.data['exp_time'][(obs.data['priority'] >= 0) & (obs.data['priority'] < 9)] / 1800))
plt.bar(np.arange(hist.size), hist)

hist = np.bincount(np.ceil(obs.data['exp_time'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)] / 1800))
plt.bar(np.arange(hist.size), hist, color='r')

plt.title('Distribution of required visit')

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 4], wspace=0.4)   

axs = [None, None, None]
ax = axs[0] = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')

ax = axs[1] = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')

ax = axs[2] = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')

cmd.plot_observation(axs[0], obs, c=obs.data['priority'][obs_assigned], mask=obs_assigned, cmap=cmap)
ccd.plot_observation(axs[1], obs, c=obs.data['priority'][obs_assigned], mask=obs_assigned, cmap=cmap)
l = fov.plot_observation(axs[2], obs, c=obs.data['priority'][obs_assigned], size=0.5, mask=obs_assigned, cmap=cmap)

f.colorbar(l, ax=axs[2], label='target priority')

f.suptitle(f'Observed stars')

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')

for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        fov.plot_observation(ax, sky, c='b', mask=sky_assignments[vidx], size=0.5, cmap=cmap)
        fov.plot_observation(ax, fluxstd, c='r', mask=fluxstd_assignments[vidx], size=0.5, cmap=cmap)
        fov.plot_observation(ax, obs, c='k', mask=obs_assignments[vidx], size=0.2, cmap=cmap)

f.suptitle(f'Fiber assignments')

In [0]:
cmap = plt.get_cmap('tab10')

for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        f = plt.figure(figsize=(8, 3), dpi=240)
        gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

        ax = f.add_subplot(gs[0])
        cmd.plot_observation(ax, obs, c='lightgray')
        cmd.plot_observation(ax, obs, c=obs.data['priority'][obs_assignments[vidx]], mask=obs_assignments[vidx], cmap=cmap)

        ax = f.add_subplot(gs[1])
        ccd.plot_observation(ax, obs, c='lightgray')
        ccd.plot_observation(ax, obs, c=obs.data['priority'][obs_assignments[vidx]], mask=obs_assignments[vidx], cmap=cmap)

        ax = f.add_subplot(gs[2], projection=wcs.wcs)
        fov.plot_observation(ax, obs, c='lightgray')
        fov.plot_observation(ax, obs, c=obs.data['priority'][obs_assignments[vidx]], mask=obs_assignments[vidx], cmap=cmap)

        f.suptitle(f'Pointing {pidx}, Visit {vidx}')

In [0]:
cmap = plt.get_cmap('tab10')

for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        f = plt.figure(figsize=(8, 3), dpi=240)
        gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

        ax = f.add_subplot(gs[2], projection=wcs.wcs)
        fov.plot_observation(ax, obs, c='lightgray')
        fov.plot_catalog(ax, sky, c='b', mask=sky_assignments[vidx], size=2, cmap=cmap)
        fov.plot_catalog(ax, fluxstd, c='r', mask=fluxstd_assignments[vidx], size=2, cmap=cmap)
        fov.plot_observation(ax, obs, c='k', mask=obs_assignments[vidx], size=0.5, cmap=cmap)

        f.suptitle(f'Pointing {pidx}, Visit {vidx}')

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   
ax = f.add_subplot(gs[2], projection=wcs.wcs)

fov.plot_observation(ax, obs, c='lightgray')

for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):

        fov.plot_catalog(ax, sky, c='b', mask=sky_assignments[vidx], size=0.5, cmap=cmap)
        fov.plot_catalog(ax, fluxstd, c='r', mask=fluxstd_assignments[vidx], size=0.5, cmap=cmap)
        fov.plot_observation(ax, obs, c='k', mask=obs_assignments[vidx], size=0.5, cmap=cmap)

        # f.suptitle(f'Pointing {pidx}, Visit {vidx}')